In [ ]:
# Install libraries
!pip install pesq pystoi speechbrain jiwer ace-tools

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 42.9 MB/s eta 0:00:00
  Created wheel for pesq: filename=pesq-0.0.4-cp311-cp311-linux_x86_64.whl size=275947 sha256=f71d1171043fe02ae78126e413ef774dcf54942dd3dafbf4edbdfc5b678d42f2
  Stored in directory: /root/.cache/pip/wheels/ae/f1/23/2698d0bf31eec2b2aa50623b5d93b6206c49c7155d0e31345d
Successfully built pesq


In [ ]:
import librosa
import librosa.display
import numpy as np
import soundfile as sf
from scipy.linalg import svd
from scipy.signal import butter, sosfilt
from pesq import pesq
from pystoi import stoi
from speechbrain.inference.ASR import StreamingASR
import os
import glob
import pandas as pd
import re
from jiwer import wer
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import pandas as pd


In [ ]:
from speechbrain.utils.dynamic_chunk_training import DynChunkTrainConfig

dynchunktrain_config = DynChunkTrainConfig(
    chunk_size=1024,
    left_context_size=5
)


In [ ]:
# load the model
asr = StreamingASR.from_hparams(
    source="speechbrain/asr-streaming-conformer-librispeech",
    savedir="pretrained_models/asr-streaming-conformer-librispeech"
)

# paths
base = "/content/drive/MyDrive/SpeechProcessing/"
noisy_dirs = [base + f"NoisySpeech0{i}/" if i else base + "NoisySpeech/" for i in range(6)]
clean_dir = base + "CleanTranscriptions/"
os.makedirs(clean_dir, exist_ok=True)

# transcriber
def transcribe(x, sr):
    tmp = "temp.wav"
    sf.write(tmp, x, sr)
    txt = asr.transcribe_file(tmp)
    os.remove(tmp)
    return txt

wer_log = []

# go folder by folder
for idx, folder in enumerate(noisy_dirs):
    trans_dir = os.path.join(folder, "Transcriptions")
    wer_dir = os.path.join(folder, "Noisy_WER")
    os.makedirs(trans_dir, exist_ok=True)
    os.makedirs(wer_dir, exist_ok=True)

    # get clean txt for this batch
    clean_path = os.path.join(clean_dir, f"CleanTranscription_{idx+1}.txt")
    with open(clean_path, "r") as f:
        ref = f.read().strip()

    files = glob.glob(os.path.join(folder, "*.wav"))

    for fpath in files:
        fname = os.path.basename(fpath).replace(".wav", "")
        y, sr = librosa.load(fpath, sr=16000)

        # transcribe
        hyp = transcribe(y, sr)

        # save txt
        out_txt = os.path.join(trans_dir, f"NoisyTranscription_{fname}.txt")
        with open(out_txt, "w") as f:
            f.write(hyp)

        # calc WER
        w = wer(ref, hyp)

        # save WER txt
        wer_path = os.path.join(wer_dir, f"WER_Noisy_{fname}.txt")
        with open(wer_path, "w") as f:
            f.write(f"WER for {fname}: {w:.4f}")

        # stash result
        wer_log.append({
            "File": fname,
            "WER (Noisy)": w
        })

# dump csv of all WERs
df = pd.DataFrame(wer_log)
df.to_csv(os.path.join(base, "NoisyWER/WER_Noisy_Results.csv"), index=False)

print("done w/ noisy transcriptions + WERs ✅")


hyperparams.yaml:   0%|          | 0.00/6.02k [00:00<?, ?B/s]

model.ckpt:   0%|          | 0.00/334M [00:00<?, ?B/s]

normalizer.ckpt:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

tokenizer.ckpt:   0%|          | 0.00/253k [00:00<?, ?B/s]

OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/SpeechProcessing/NoisyWER'

In [ ]:
# base folder
base = "/content/drive/MyDrive/SpeechProcessing/"
noisy_dirs = sorted(glob.glob(os.path.join(base, "NoisySpeech*")))
wer_data = []

# loop thru all the NoisySpeechXX folders
for folder in noisy_dirs:
    wer_dir = os.path.join(folder, "Noisy_WER")
    if not os.path.exists(wer_dir):
        print(f"[skip] no WER folder in {folder}")
        continue

    txts = glob.glob(os.path.join(wer_dir, "WER_Noisy_*.txt"))

    for f in txts:
        name = os.path.basename(f).replace("WER_Noisy_", "").replace(".txt", "")
        with open(f, "r") as file:
            txt = file.read().strip()

        m = re.search(r"([\d.]+)", txt)
        if not m:
            print(f"[warn] can't parse WER in {f}")
            continue

        wer_val = float(m.group(1))
        wer_data.append({"File": name, "WER (Noisy)": wer_val})

# to csv
out_path = os.path.join(base, "WER_Results/WER_Noisy_Results.csv")
os.makedirs(os.path.dirname(out_path), exist_ok=True)

pd.DataFrame(wer_data).to_csv(out_path, index=False)

print(f"[done] WERs saved → {out_path}")


WER results saved to /content/drive/MyDrive/SpeechProcessing/WER_Results/WER_Noisy_Results.csv


In [ ]:
# folders
noisy_dirs = [
    base + "NoisySpeech/",
    base + "NoisySpeech02/",
    base + "NoisySpeech03/",
    base + "NoisySpeech04/",
    base + "NoisySpeech05/",
    base + "NoisySpeech06/"
]
thresh_dir = base + "OptimalThresholds/"
os.makedirs(thresh_dir, exist_ok=True)

# quick rank finder
def get_svd_thresh(x, sr):
    spec = librosa.stft(x)
    U, S, V = np.linalg.svd(spec, full_matrices=False)
    energy = np.cumsum(S**2) / np.sum(S**2)
    k = np.argmax(energy >= 0.95)
    return k

# results holder
thresh_log = []

# go thru each folder
for d in noisy_dirs:
    files = librosa.util.find_files(d, ext=["wav"])
    for f in files:
        name = os.path.basename(f).replace(".wav", "")
        x, sr = librosa.load(f, sr=16000)

        k = get_svd_thresh(x, sr)

        # drop individual txt
        out_txt = os.path.join(thresh_dir, f"Threshold_{name}.txt")
        with open(out_txt, "w") as w:
            w.write(str(k))

        thresh_log.append({"File": name, "Optimal Threshold": k})

# dump all thresholds
pd.DataFrame(thresh_log).to_csv(os.path.join(thresh_dir, "SVD_Thresholds.csv"), index=False)

print("thresholds saved.")


Optimal SVD thresholds computed and saved successfully.


In [ ]:
# folders
noisy_dirs = [
    base + "NoisySpeech/",
    base + "NoisySpeech02/",
    base + "NoisySpeech03/",
    base + "NoisySpeech04/",
    base + "NoisySpeech05/",
    base + "NoisySpeech06/"
]
thresh_dir = base + "OptimalThresholds/"
denoised_base = base + "DenoisedSpeech/"
os.makedirs(denoised_base, exist_ok=True)

# svd cleaner
def svd_denoise(x, sr, k):
    spec = librosa.stft(x)
    U, S, V = np.linalg.svd(spec, full_matrices=False)
    S[k:] = 0
    cleaned = U @ np.diag(S) @ V
    return librosa.istft(cleaned)

# loop each folder
for idx, d in enumerate(noisy_dirs):
    out_dir = os.path.join(denoised_base, f"DenoisedSpeech{idx+1}")
    os.makedirs(out_dir, exist_ok=True)

    files = librosa.util.find_files(d, ext=["wav"])
    for f in files:
        name = os.path.basename(f)
        x, sr = librosa.load(f, sr=16000)

        # load threshold
        t_path = os.path.join(thresh_dir, f"Threshold_{name.replace('.wav', '')}.txt")
        if not os.path.exists(t_path):
            print(f"no threshold for {name}, skip")
            continue

        with open(t_path, "r") as t:
            k = int(t.read().strip())

        # apply cleanup
        x_d = svd_denoise(x, sr, k)

        # save it
        sf.write(os.path.join(out_dir, name), x_d, sr)

print("denoised audio done.")


Denoised speech files saved successfully.


In [ ]:
# paths
denoised_txt_dir = base + "DenoisedTranscriptions/"
denoised_wer_dir = base + "WER_Results/"
os.makedirs(denoised_txt_dir, exist_ok=True)
os.makedirs(denoised_wer_dir, exist_ok=True)

wer_log = []

# loop through each batch folder
for i, noisy_dir in enumerate(noisy_dirs):
    # clean ref
    ref_path = os.path.join(clean_dir, f"CleanTranscription_{i+1}.txt")
    with open(ref_path, "r") as f:
        ref = f.read().strip()

    # threshold lookup
    thresh_csv = os.path.join(base, "OptimalThresholds", f"OptimalThresholds_{i+1}.csv")
    thresh_df = pd.read_csv(thresh_csv, index_col="File")

    # denoised files
    denoised_dir = os.path.join(denoised_base, f"DenoisedSpeech{i+1}")
    denoised_files = glob.glob(os.path.join(denoised_dir, "*.wav"))

    for fpath in denoised_files:
        name = os.path.basename(fpath).replace(".wav", "")
        x, sr = librosa.load(fpath, sr=16000)

        # transcribe
        hyp = transcribe(x, sr)

        # save transcription
        out_txt = os.path.join(denoised_txt_dir, f"DenoisedTranscription_{name}.txt")
        with open(out_txt, "w") as w:
            w.write(hyp)

        # calc WER
        w_score = wer(ref, hyp)

        # save WER txt
        wer_txt = os.path.join(denoised_wer_dir, f"WER_Denoised_{name}.txt")
        with open(wer_txt, "w") as w:
            w.write(f"WER for {name}: {w_score:.4f}")

        # save for master csv
        wer_log.append({
            "File": name,
            "WER (Denoised)": w_score,
            "Optimal Threshold": thresh_df.loc[name, "Optimal Threshold"]
        })

# final CSV drop
df = pd.DataFrame(wer_log)
df.to_csv(os.path.join(denoised_wer_dir, "WER_Denoised_Results.csv"), index=False)

print("denoised WERs and transcripts saved.")


In [ ]:
# output dir
den_wer_dir = os.path.join(base, "DenoisedWER/")
os.makedirs(den_wer_dir, exist_ok=True)

# folders
den_dirs = [os.path.join(base, f"DenoisedSpeech{i}/") for i in range(1, 7)]
wer_log = []

# loop each folder
for i, den_dir in enumerate(den_dirs):
    txt_dir = os.path.join(den_dir, "Denoised_Transcriptions")
    wer_dir = os.path.join(den_dir, "Denoised_WER")
    os.makedirs(txt_dir, exist_ok=True)
    os.makedirs(wer_dir, exist_ok=True)

    ref_path = os.path.join(clean_dir, f"CleanTranscription_{i+1}.txt")
    with open(ref_path, "r") as f:
        ref = f.read().strip()

    files = glob.glob(os.path.join(den_dir, "*.wav"))
    for fpath in files:
        name = os.path.basename(fpath).replace(".wav", "")
        x, sr = librosa.load(fpath, sr=16000)

        hyp = transcribe(x, sr)

        # drop txt
        out_txt = os.path.join(txt_dir, f"DenoisedTranscription_{name}.txt")
        with open(out_txt, "w") as w:
            w.write(hyp)

        # calc WER
        w_score = wer(ref, hyp)

        # save WER
        wer_txt = os.path.join(wer_dir, f"WER_Denoised_{name}.txt")
        with open(wer_txt, "w") as w:
            w.write(f"WER for {name}: {w_score:.4f}")

        wer_log.append({
            "File": name,
            "WER (Denoised)": w_score
        })

# all WERs → csv
pd.DataFrame(wer_log).to_csv(os.path.join(den_wer_dir, "WER_Denoised_Results.csv"), index=False)

print("denoised transcriptions + WERs done.")


Denoised speech transcriptions and WER results saved successfully.


In [ ]:
# output path for summary
os.makedirs(os.path.join(base, "WER_Results"), exist_ok=True)

# grab folders
den_folders = sorted(glob.glob(os.path.join(base, "DenoisedSpeech*")))
wer_log = []

# go thru folders starting at DenoisedSpeech3/
for i, folder in enumerate(den_folders[2:], start=2):
    txt_dir = os.path.join(folder, "Transcriptions")
    wer_dir = os.path.join(folder, "Denoised_WER")
    os.makedirs(txt_dir, exist_ok=True)
    os.makedirs(wer_dir, exist_ok=True)

    # clean reference
    ref_path = os.path.join(base, "CleanTranscriptions", f"CleanTranscription_{i+1}.txt")
    with open(ref_path, "r") as f:
        ref = f.read().strip()

    files = glob.glob(os.path.join(folder, "*.wav"))
    for fpath in files:
        name = os.path.basename(fpath).replace(".wav", "")
        x, sr = librosa.load(fpath, sr=16000)

        # transcribe
        hyp = transcribe(x, sr)

        # save txt
        out_txt = os.path.join(txt_dir, f"DenoisedTranscription_{name}.txt")
        with open(out_txt, "w") as w:
            w.write(hyp)

        # WER
        score = wer(ref, hyp)
        out_wer = os.path.join(wer_dir, f"WER_Denoised_{name}.txt")
        with open(out_wer, "w") as w:
            w.write(f"WER for {name}: {score:.4f}")

        wer_log.append({
            "File": name,
            "WER (Denoised)": score
        })

# dump final CSV
pd.DataFrame(wer_log).to_csv(os.path.join(base, "WER_Results", "WER_Denoised_Results.csv"), index=False)

print("denoised transcriptions + WERs saved.")


hyperparams.yaml:   0%|          | 0.00/6.02k [00:00<?, ?B/s]

model.ckpt:   0%|          | 0.00/334M [00:00<?, ?B/s]

normalizer.ckpt:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

tokenizer.ckpt:   0%|          | 0.00/253k [00:00<?, ?B/s]

Denoised speech transcriptions and WER results saved successfully.


In [ ]:
# WER output folder
os.makedirs(os.path.join(base, "WER_Results"), exist_ok=True)

# folders
noisy_dirs = sorted(glob.glob(os.path.join(base, "NoisySpeech*")))
den_dirs = sorted(glob.glob(os.path.join(base, "DenoisedSpeech*")))

# sanity check
if len(noisy_dirs) != len(den_dirs):
    print("mismatch in # of noisy vs denoised folders")

wer_log = []

# loop through paired dirs
for i, den_dir in enumerate(den_dirs):
    try:
        noisy_dir = noisy_dirs[i]
    except IndexError:
        print(f"no noisy match for {den_dir}")
        continue

    noisy_txt_dir = os.path.join(noisy_dir, "Transcriptions")
    if not os.path.exists(noisy_txt_dir):
        print(f"no transcriptions in {noisy_dir}, skip")
        continue

    den_txt_dir = os.path.join(den_dir, "Transcriptions")
    wer_out_dir = os.path.join(den_dir, "WER_Noisy_vs_Denoised")
    os.makedirs(wer_out_dir, exist_ok=True)

    den_files = glob.glob(os.path.join(den_txt_dir, "*.txt"))
    for f in den_files:
        name = os.path.basename(f).replace("DenoisedTranscription_", "").replace(".txt", "")
        noisy_file = os.path.join(noisy_txt_dir, f"NoisyTranscription_{name}.txt")

        if not os.path.exists(noisy_file):
            print(f"no match for {name} in noisy, skip")
            continue

        with open(noisy_file, "r") as n:
            noisy_txt = n.read().strip()
        with open(f, "r") as d:
            den_txt = d.read().strip()

        score = wer(noisy_txt, den_txt)

        out_txt = os.path.join(wer_out_dir, f"WER_Noisy_vs_Denoised_{name}.txt")
        with open(out_txt, "w") as w:
            w.write(f"WER (Noisy vs Denoised) for {name}: {score:.4f}")

        wer_log.append({
            "File": name,
            "WER (Noisy vs. Denoised)": score
        })

# final dump
df = pd.DataFrame(wer_log)
df.to_csv(os.path.join(base, "WER_Results", "WER_Noisy_vs_Denoised.csv"), index=False)

print("noisy vs denoised WERs saved.")


WER (Noisy vs. Denoised) results saved successfully.


In [ ]:
# paths
base = "/content/drive/MyDrive/SpeechProcessing/WER_Results/"

# load all 3 WER tables
noisy_df = pd.read_csv(base + "WER_Noisy_Results.csv")        # clean vs noisy
den_df = pd.read_csv(base + "WER_Denoised_Results.csv")       # clean vs denoised
vs_df = pd.read_csv(base + "WER_Noisy_vs_Denoised.csv")       # noisy vs denoised

# quick look
print("Noisy vs Denoised:")
display(vs_df.head())

print("\nClean vs Noisy:")
display(noisy_df.head())

print("\nClean vs Denoised:")
display(den_df.head())


In [ ]:
# mean WERs
mean_noisy = noisy_df["WER (Noisy)"].mean()
mean_den = den_df["WER (Denoised)"].mean()
mean_vs = vs_df["WER (Noisy vs. Denoised)"].mean()

# % diffs
imp_clean_den = ((mean_noisy - mean_den) / mean_noisy) * 100
diff_vs = ((mean_noisy - mean_vs) / mean_noisy) * 100

# print results
print("avg WERs:")
print(f"Clean vs Noisy:     {mean_noisy:.4f}")
print(f"Clean vs Denoised:  {mean_den:.4f}")
print(f"Noisy vs Denoised:  {mean_vs:.4f}")

print("\nrelative change:")
print(f"↓ WER drop after denoising:      {imp_clean_den:.2f}%")
print(f"↔ difference between noisy/den:  {diff_vs:.2f}%")


In [ ]:
# bar chart for WER comparison
labels = ["Clean vs Noisy", "Clean vs Denoised", "Noisy vs Denoised"]
scores = [mean_noisy, mean_den, mean_vs]

plt.figure(figsize=(8, 5))
plt.bar(labels, scores, color=["red", "green", "blue"])
plt.ylabel("WER")
plt.ylim(0, 1)
plt.title("WER Comparison")
plt.tight_layout()
plt.show()


In [ ]:
# folders
noisy_dirs = [
    base + "NoisySpeech/",
    base + "NoisySpeech02/",
    base + "NoisySpeech03/",
    base + "NoisySpeech04/",
    base + "NoisySpeech05/",
    base + "NoisySpeech06/"
]
den_dirs = [
    base + "DenoisedSpeech/DenoisedSpeech1/",
    base + "DenoisedSpeech/DenoisedSpeech2/",
    base + "DenoisedSpeech/DenoisedSpeech3/",
    base + "DenoisedSpeech/DenoisedSpeech4/",
    base + "DenoisedSpeech/DenoisedSpeech5/",
    base + "DenoisedSpeech/DenoisedSpeech6/"
]

# parse noise type + SNR
def get_noise_info(fname):
    m = re.search(r"with_([A-Za-z]+)_\d+_SNR_(-?\d+)dB", fname)
    return (m.group(1), int(m.group(2))) if m else (None, None)

# extract WER score from file
def read_wer(path):
    with open(path, "r") as f:
        txt = f.read().strip()
    m = re.search(r"WER.*?:\s*([\d.]+)", txt)
    return float(m.group(1)) if m else None

# grab noisy WERs
noisy_data = {}
for folder in noisy_dirs:
    files = glob.glob(os.path.join(folder, "Noisy_WER", "*.txt"))
    for f in files:
        fname = os.path.basename(f)
        ntype, snr = get_noise_info(fname)
        if ntype:
            noisy_data[fname] = {
                "Noise Type": ntype,
                "Noise Level (dB)": snr,
                "WER Noisy": read_wer(f)
            }

# grab denoised WERs
denoised_data = {}
for folder in den_dirs:
    files = glob.glob(os.path.join(folder, "Denoised_WER", "*.txt"))
    for f in files:
        fname = os.path.basename(f).replace("WER_Denoised_", "WER_Noisy_")
        denoised_data[fname] = read_wer(f)

# quick match check (first 5)
print("\nChecking file matches:")
for i, fname in enumerate(noisy_data.keys()):
    if fname in denoised_data:
        print(f"[ok] {fname}")
    else:
        print(f"[miss] {fname}")
    if i == 4:
        break


In [ ]:
# folders
noisy_dirs = [
    base + "NoisySpeech/",
    base + "NoisySpeech02/",
    base + "NoisySpeech03/",
    base + "NoisySpeech04/",
    base + "NoisySpeech05/",
    base + "NoisySpeech06/"
]
den_dirs = [
    base + "DenoisedSpeech/DenoisedSpeech1/",
    base + "DenoisedSpeech/DenoisedSpeech2/",
    base + "DenoisedSpeech/DenoisedSpeech3/",
    base + "DenoisedSpeech/DenoisedSpeech4/",
    base + "DenoisedSpeech/DenoisedSpeech5/",
    base + "DenoisedSpeech/DenoisedSpeech6/"
]

# pull info from filename
def get_info(fname):
    m = re.search(r"with_([A-Za-z]+)_\d+_SNR_(-?\d+)dB", fname)
    return (m.group(1), int(m.group(2))) if m else (None, None)

# get WER from .txt
def get_wer(path):
    with open(path, "r") as f:
        m = re.search(r"WER.*?:\s*([\d.]+)", f.read())
        return float(m.group(1)) if m else None

# unify filename keys
def norm(fname):
    return fname.replace("WER_Noisy_", "").replace("WER_Denoised_", "")

# collect noisy WERs
noisy_data = {}
for folder in noisy_dirs:
    files = glob.glob(os.path.join(folder, "Noisy_WER", "*.txt"))
    for f in files:
        name = norm(os.path.basename(f))
        ntype, snr = get_info(f)
        if ntype:
            noisy_data[name] = {
                "Noise Type": ntype,
                "Noise Level (dB)": snr,
                "WER Noisy": get_wer(f)
            }

# collect denoised WERs
den_data = {}
for folder in den_dirs:
    files = glob.glob(os.path.join(folder, "Denoised_WER", "*.txt"))
    for f in files:
        name = norm(os.path.basename(f))
        ntype, snr = get_info(f)
        if ntype:
            den_data[name] = {
                "Noise Type": ntype,
                "Noise Level (dB)": snr,
                "WER Denoised": get_wer(f)
            }

# combine matched entries
combined = []
for name in noisy_data:
    if name in den_data:
        combined.append({
            "Noise Type": noisy_data[name]["Noise Type"],
            "Noise Level (dB)": noisy_data[name]["Noise Level (dB)"],
            "WER Noisy": noisy_data[name]["WER Noisy"],
            "WER Denoised": den_data[name]["WER Denoised"]
        })

# df and save
df = pd.DataFrame(combined)
out = os.path.join(base, "Formatted_WER_Results.csv")
df.to_csv(out, index=False)

print(df.head())
print(f"formatted WERs saved to: {out}")


In [ ]:
# load formatted WERs
df = pd.read_csv(base + "Formatted_WER_Results.csv")

# avg across noise level + type
avg = df.groupby(["Noise Level (dB)", "Noise Type"])[["WER Noisy", "WER Denoised"]].mean().reset_index()

# reshape for plotting
melted = avg.melt(
    id_vars=["Noise Level (dB)", "Noise Type"],
    value_vars=["WER Noisy", "WER Denoised"],
    var_name="Type",
    value_name="WER"
)

# plot
plt.figure(figsize=(10, 6))
sns.lineplot(
    data=melted,
    x="Noise Level (dB)",
    y="WER",
    hue="Noise Type",
    style="Type",
    markers=True,
    dashes={"WER Noisy": (4, 2), "WER Denoised": ""}
)

plt.axhline(0, color="red", linestyle="--", lw=1)
plt.title("WER by Noise Type + SNR")
plt.xlabel("SNR (dB)")
plt.ylabel("WER")
plt.legend(title="Noise / Type")
plt.grid(True)
plt.tight_layout()
plt.show()
